*`Nom & Prenom : GUEYE DIE <br> Classe: M1 BI <br> Code: 220642

## Importation des modules Spark necessaires

In [312]:
import org.apache.log4j.{Level, Logger}
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.10.9` // Not required since almond 0.7.0 (will be automatically added when importing spark)
import org.apache.log4j.{Level, Logger}
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import org.apache.spark.sql._
import org.apache.spark.sql.{SparkSession, functions}
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions.{col, translate}
import org.apache.spark.sql.functions._

import org.apache.log4j.{Level, Logger}

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                // Not required since almond 0.7.0 (will be automatically added when importing spark)

import org.apache.log4j.{Level, Logger}

import $ivy.$                                    

import org.apache.spark.sql._

import org.apache.spark.sql.{SparkSession, functions}

import org.apache.spark.sql.{SparkSession, DataFrame}

import org.apache.spark.sql.functions.{col, translate}

import org.apache.spark.sql.functions._

In [60]:
val rootLogger = Logger.getRootLogger()
rootLogger.setLevel(Level.ERROR)

Logger.getLogger("org.apache.spark").setLevel(Level.WARN)
Logger.getLogger("org.spark-project").setLevel(Level.WARN)

rootLogger: Logger = org.apache.log4j.spi.RootLogger@3d90ee57

## Spark Session

In [3]:
Logger.getLogger("org").setLevel(Level.OFF)

In [3]:
val spark = {
  SparkSession.builder()
    .master("local")
    .appName("BD-FS FIRE")
    .config("spark.some.option.config", "config-value")
    .getOrCreate()
}

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
23/05/26 11:39:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


spark: SparkSession = org.apache.spark.sql.SparkSession@c2a16d1

## Pré-traitement

### Chargement des données de Janvier - Décembre 

In [4]:
val dfJanuary = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_January_2019.csv")
val dfFebruary = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_February_2019.csv")
val dfMarch = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_March_2019.csv")
val dfApril = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_April_2019.csv")
val dfMay = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_May_2019.csv")
val dfJune = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_June_2019.csv")
val dfJuly = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_July_2019.csv")
val dfAugust = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_August_2019.csv")
val dfSeptembre = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_September_2019.csv")
val dfOctober = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_October_2019.csv")
val dfNovember = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_November_2019.csv")
val dfDecember = spark.read.format("csv").option("header", "true").load("data/Vente/Sales_December_2019.csv")

dfJanuary: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfFebruary: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfMarch: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfApril: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfMay: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfJune: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfJuly: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfAugust: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfSeptembre: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfOctober: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfNovember: DataFrame = [Order ID: string, Product: string ... 4 more fields]
dfDecember: DataFrame = [Order ID: string, Product: string ... 4 more fields]

### Regrouper les DataFrames en un seul DataFrame

In [93]:
val newdf: DataFrame = dfJanuary
                        .union(dfFebruary)
                        .union(dfMarch)
                        .union(dfApril)
                        .union(dfMay)
                        .union(dfJune)
                        .union(dfJuly)
                        .union(dfAugust)
                        .union(dfSeptembre)
                        .union(dfOctober)
                        .union(dfNovember)
                        .union(dfDecember)
newdf2.show()

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  141234|              iPhone|               1|       700|01/22/19 21:25|944 Walnut St, Bo...|
|  141235|Lightning Chargin...|               1|     14.95|01/28/19 14:15|185 Maple St, Por...|
|  141236|    Wired Headphones|               2|     11.99|01/17/19 13:33|538 Adams St, San...|
|  141237|    27in FHD Monitor|               1|    149.99|01/05/19 20:33|738 10th St, Los ...|
|  141238|    Wired Headphones|               1|     11.99|01/25/19 11:59|387 10th St, Aust...|
|  141239|AAA Batteries (4-...|               1|      2.99|01/29/19 20:22|775 Willow St, Sa...|
|  141240|27in 4K Gaming Mo...|               1|    389.99|01/26/19 12:16|979 Park St, Los ...|
|  141241|USB-C Charging Cable|         

newdf: DataFrame = [Order ID: string, Product: string ... 4 more fields]

In [94]:
newdf.printSchema

root
 |-- Order ID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: string (nullable = true)
 |-- Price Each: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



Nous constatons que tout les  variables sont de type string alors que certains devrait etre de type double ou Integer

### Convertir les variables Order ID, Quantity Ordered, et Price Each en type Double

In [76]:
//Selectionner les variables qui doit etre de type double
val TypeDouble = newdf2.select(
    "Price Each",
    "Quantity Ordered",
    "Order ID"
  )

//Convertir ces variables en type double
import org.apache.spark.sql.types.StringType

val StringColumns = for (x <- TypeDouble.schema.fields if (x.dataType == StringType)) yield x.name  
var doublesDF = newdf2

for (c <- StringColumns)
  doublesDF = doublesDF.withColumn(c, col(c).cast("double"))

val columns = StringColumns.mkString("\n - ")
println(s"Columns converted from Integer to Double:\n - $columns \n")
println("*-"*80)

Columns converted from Integer to Double:
 - Price Each
 - Quantity Ordered
 - Order ID 

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


TypeDouble: DataFrame = [Price Each: string, Quantity Ordered: string ... 1 more field]
import org.apache.spark.sql.types.StringType


StringColumns: Array[String] = Array(
  "Price Each",
  "Quantity Ordered",
  "Order ID"
)
doublesDF: DataFrame = [Order ID: double, Product: string ... 6 more fields]
columns: String = """Price Each
 - Quantity Ordered
 - Order ID"""

In [95]:
doublesDF.printSchema

root
 |-- Order ID: double (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: double (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)



In [96]:
val newdf = doublesDF.withColumn("Total Sales", functions.col("Quantity Ordered") * functions.col("Price Each"))
newdf.show()

+--------+--------------------+----------------+----------+--------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|Total Sales|
+--------+--------------------+----------------+----------+--------------+--------------------+-----------+
|141234.0|              iPhone|             1.0|     700.0|01/22/19 21:25|944 Walnut St, Bo...|      700.0|
|141235.0|Lightning Chargin...|             1.0|     14.95|01/28/19 14:15|185 Maple St, Por...|      14.95|
|141236.0|    Wired Headphones|             2.0|     11.99|01/17/19 13:33|538 Adams St, San...|      23.98|
|141237.0|    27in FHD Monitor|             1.0|    149.99|01/05/19 20:33|738 10th St, Los ...|     149.99|
|141238.0|    Wired Headphones|             1.0|     11.99|01/25/19 11:59|387 10th St, Aust...|      11.99|
|141239.0|AAA Batteries (4-...|             1.0|      2.99|01/29/19 20:22|775 Willow St, Sa...|       2.99|
|141240.0|27in 4K Gaming Mo.

newdf: DataFrame = [Order ID: double, Product: string ... 5 more fields]

### Séparation de la colonne "Purchase Address" en 2 colonne distincte (Address et Ville)

In [97]:
val newdf1 = newdf.withColumn("Address", functions.split(functions.col("Purchase Address"), ","))
  .withColumn("Ville", functions.split(functions.col("Purchase Address"), ",") .getItem(1))

newdf1.printSchema

root
 |-- Order ID: double (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: double (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Total Sales: double (nullable = true)
 |-- Address: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Ville: string (nullable = true)



newdf1: DataFrame = [Order ID: double, Product: string ... 7 more fields]

In [85]:
// Convertir la colonne "Order Date" en format date(MM/dd/yyyy HH:mm)
val dfDate = newdf1.withColumn("OrderDate", functions.to_date(functions.col("Order Date"), "MM/dd/yyyy HH:mm"))

// Extraire le mois et l'année à partir de la colonne "OrderDate"
val dfMonthYear = dfDate.withColumn("Month", functions.month(functions.col("OrderDate")))
  .withColumn("Year", functions.year(functions.col("OrderDate")))

dfDate: DataFrame = [Order ID: double, Product: string ... 8 more fields]
dfMonthYear: DataFrame = [Order ID: double, Product: string ... 10 more fields]

## Réponses aux questions

1. Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette année-là ?

In [98]:
// Calculer le total des ventes par année
val totalSalesYearly = dfMonthYear.groupBy("Year")
  .agg(functions.sum("Total Sales").alias("TotalSales"))
  .orderBy(functions.desc("TotalSales"))

// Obtenir la meilleure année en termes de ventes
val bestYearRow = totalSalesYearly.first()
val bestYear = bestYearRow.getAs[Int]("Year")
val bestYearSale = bestYearRow.getAs[Double]("TotalSales")

// Afficher le résultat
println(s"L'année qui a été la meilleure en termes de ventes est l'année 20$bestYear.")
println(s"Le montant total des ventes pour cette année est de $bestYearSale.")

L'année qui a été la meilleure en termes de ventes est l'année 2019.
Le montant total des ventes pour cette année est de 3.3799036410004914E7.


totalSalesYearly: Dataset[Row] = [Year: int, TotalSales: double]
bestYearRow: Row = [19,3.3799036410004914E7]
bestYear: Int = 19
bestYearSale: Double = 3.3799036410004914E7

2. Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?

In [99]:
// Calculer le total des ventes par mois
val totalMonthSales = dfMonthYear.groupBy("Month", "Year")
  .agg(functions.sum("Total Sales").alias("TotalSales"))
  .orderBy(functions.desc("TotalSales"))
totalMonthSales.show()

// Obtenir le meilleur mois en termes de ventes
val bestMonthRow = totalMonthSales.first()
val bestMonth = bestMonthRow.getAs[Int]("Month")
val bestMonthSales = bestMonthRow.getAs[Double]("TotalSales")

// Afficher le résultat
println(s"Le meilleur mois en termes de ventes est le mois $bestMonth.")
println(s"Le montant total des ventes pour ce mois est de $bestMonthSales.")


+-----+----+------------------+
|Month|Year|        TotalSales|
+-----+----+------------------+
|   11|  19| 3715537.459999819|
|   10|  19| 3587664.439999961|
|    4|  19| 3320523.329999849|
|    3|  19| 3043415.159999854|
|    5|  19| 2943146.869999862|
|   12|  19|2824942.9500003685|
|    9|  19|2674799.2499998747|
|    6|  19| 2553346.229999885|
|    7|  19| 2532465.759999886|
|    2|  19| 2345457.729999894|
|    8|  19|  2192137.81999991|
|    1|  19|2065599.4099999147|
|   12|  18| 692999.5599999847|
| null|null|              null|
+-----+----+------------------+

Le meilleur mois en termes de ventes est le mois 11.
Le montant total des ventes pour ce mois est de 3715537.459999819.


totalMonthSales: Dataset[Row] = [Month: int, Year: int ... 1 more field]
bestMonthRow: Row = [11,19,3715537.459999819]
bestMonth: Int = 11
bestMonthSales: Double = 3715537.459999819

3. Quelle ville a enregistré le plus grand nombre de ventes ?

In [100]:
// Calculer le total des ventes par ville
val citySales = newdf1.groupBy("Ville")
  .agg(functions.sum("Total Sales").alias("TotalSales"))
  .orderBy(functions.desc("TotalSales"))
citySales.show()

// Obtenir la ville avec le plus grand nombre de vente
val bestCityRow = citySales.first()
val bestCity = bestCityRow.getString(0)
val bestCitySales = bestCityRow.getDouble(1)

// Afficher le résultat
println(s"La ville avec le plus grand nombre de ventes est $bestCity.")
println(s"Le montant total des ventes dans cette ville est de $bestCitySales.")

+--------------+------------------+
|         Ville|        TotalSales|
+--------------+------------------+
| San Francisco| 8262203.909999811|
|   Los Angeles| 5452570.799999968|
| New York City|  4664317.42999998|
|        Boston| 3661642.009999992|
|       Atlanta|2795498.5799999936|
|        Dallas| 2767975.399999994|
|       Seattle|2747755.4799999953|
|      Portland| 2320490.609999995|
|        Austin| 1819581.749999999|
|          null|              null|
+--------------+------------------+

La ville avec le plus grand nombre de ventes est  San Francisco.
Le montant total des ventes dans cette ville est de 8262203.909999811.


citySales: Dataset[Row] = [Ville: string, TotalSales: double]
bestCityRow: Row = [ San Francisco,8262203.909999811]
bestCity: String = " San Francisco"
bestCitySales: Double = 8262203.909999811

4. À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les clients achètent le produit ?

In [101]:
// Convertir la colonne "Order Date" en format de timestamp
val dfTimestamp = newdf1.withColumn("Timestamp", functions.to_timestamp(functions.col("Order Date"), "MM/dd/yyyy HH:mm"))

// Extraire l'heure à partir du timestamp
val dfHour = dfTimestamp.withColumn("Hour", functions.hour(functions.col("Timestamp")))

// Calcul du nombre total de Commande pour chaque heure
val hourlySales = dfHour.groupBy("Hour")
  .agg(functions.sum("Quantity Ordered").alias("QuantityOrdered"))
  .orderBy(functions.desc("QuantityOrdered"))

hourlySales.show(24)

// Obtenir l'heure avec le plus grand nombre de commandes
val bestHourRow = hourlySales.first()
val bestHour = bestHourRow.getAs[Int]("Hour")

// Afficher le résultat
println(s"L'heure optimale pour diffuser des publicités est à $bestHour heure.")

+----+---------------+
|Hour|QuantityOrdered|
+----+---------------+
|  19|        14470.0|
|  12|        14202.0|
|  11|        14005.0|
|  18|        13802.0|
|  20|        13768.0|
|  13|        13685.0|
|  14|        12362.0|
|  10|        12308.0|
|  21|        12244.0|
|  17|        12229.0|
|  16|        11662.0|
|  15|        11391.0|
|  22|         9899.0|
|   9|         9816.0|
|  23|         7065.0|
|   8|         7002.0|
|   7|         4556.0|
|   0|         4428.0|
|   6|         2810.0|
|   1|         2619.0|
|   5|         1493.0|
|   2|         1398.0|
|   4|          937.0|
|   3|          928.0|
+----+---------------+
only showing top 24 rows

L'heure optimale pour diffuser des publicités est à 19 heure.


dfTimestamp: DataFrame = [Order ID: double, Product: string ... 8 more fields]
dfHour: DataFrame = [Order ID: double, Product: string ... 9 more fields]
hourlySales: Dataset[Row] = [Hour: int, QuantityOrdered: double]
bestHourRow: Row = [19,14470.0]
bestHour: Int = 19

5. Quels produits sont le plus souvent vendus ensemble ?

In [318]:
import spark.implicits._
val dfPair = dfTimestamp.groupBy("Order ID")
  .agg(collect_set("Product").as("Products"))


//les paires de produits pour chaque commande
val Pairs = dfPair.flatMap { row =>
  val products = row.getAs[Seq[String]]("Products")
  products.combinations(2).map(pair => (pair(0), pair(1)))
}.toDF("Produit 1", "Produit 2")


// Le nombre d'occurrences des paires de produits
val PairCounts = Pairs.groupBy("Produit 1", "Produit 2")
  .count()
  .withColumn("Occurrences", col("count").cast("Int"))
  .drop("count")


// Trier
val result = PairCounts.orderBy(col("Occurrences").desc)

// Afficher le résultat 
result.show()

+--------------------+--------------------+-----------+
|           Produit 1|           Produit 2|Occurrences|
+--------------------+--------------------+-----------+
|Lightning Chargin...|              iPhone|       1011|
|USB-C Charging Cable|        Google Phone|        997|
|              iPhone|    Wired Headphones|        462|
|    Wired Headphones|        Google Phone|        422|
|Apple Airpods Hea...|              iPhone|        373|
|     Vareebadd Phone|USB-C Charging Cable|        368|
|        Google Phone|Bose SoundSport H...|        228|
|USB-C Charging Cable|    Wired Headphones|        203|
|     Vareebadd Phone|    Wired Headphones|        149|
|Lightning Chargin...|    Wired Headphones|        129|
|Lightning Chargin...|AA Batteries (4-p...|        106|
|USB-C Charging Cable|Bose SoundSport H...|        102|
|Lightning Chargin...|USB-C Charging Cable|        100|
|Apple Airpods Hea...|    Wired Headphones|        100|
|AAA Batteries (4-...|USB-C Charging Cable|     

import spark.implicits._

dfPair: DataFrame = [Order ID: double, Products: array<string>]
Pairs: DataFrame = [Produit 1: string, Produit 2: string]
PairCounts: DataFrame = [Produit 1: string, Produit 2: string ... 1 more field]
result: Dataset[Row] = [Produit 1: string, Produit 2: string ... 1 more field]

6. Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ?

In [102]:
// Calculer le nombre total de ventes pour chaque produit
val productSales = newdf1.groupBy("Product")
  .agg(functions.sum("Quantity Ordered").alias("QuantityOrdered"))
  .orderBy(functions.desc("QuantityOrdered"))
productSales.show()

// Obtenir le produit avec le plus grand nombre de commandes
val bestProductRow = productSales.first()
val bestProduct = bestProductRow.getString(0)
val bestProductSale = bestProductRow.getDouble(1)

// Afficher le résultat
println(s"Le produit le plus vendu est: $bestProduct.")

+--------------------+---------------+
|             Product|QuantityOrdered|
+--------------------+---------------+
|AAA Batteries (4-...|        31017.0|
|AA Batteries (4-p...|        27635.0|
|USB-C Charging Cable|        23975.0|
|Lightning Chargin...|        23217.0|
|    Wired Headphones|        20557.0|
|Apple Airpods Hea...|        15661.0|
|Bose SoundSport H...|        13457.0|
|    27in FHD Monitor|         7550.0|
|              iPhone|         6849.0|
|27in 4K Gaming Mo...|         6244.0|
|34in Ultrawide Mo...|         6199.0|
|        Google Phone|         5532.0|
|       Flatscreen TV|         4819.0|
|  Macbook Pro Laptop|         4728.0|
|     ThinkPad Laptop|         4130.0|
|        20in Monitor|         4129.0|
|     Vareebadd Phone|         2068.0|
|  LG Washing Machine|          666.0|
|            LG Dryer|          646.0|
|                null|           null|
+--------------------+---------------+
only showing top 20 rows

Le produit le plus vendu est: AAA Batt

productSales: Dataset[Row] = [Product: string, QuantityOrdered: double]
bestProductRow: Row = [AAA Batteries (4-pack),31017.0]
bestProduct: String = "AAA Batteries (4-pack)"
bestProductSale: Double = 31017.0

7. Quelle est la probabilité que les prochains clients commandent un câble de chargement USB-C ?

In [18]:
// Filtrer les commandes de câbles de chargement USB-C
val usbOrders = newdf1.filter(newdf1("Product") === "USB-C Charging Cable").count

// Calculer le nombre total de commandes
val totalOrders = newdf1.count()

// Calculer la probabilité
val probabilityOrderUsb = usbOrders.toDouble / totalOrders.toDouble

// Afficher la probabilité
println(s"La probabilité que les prochains clients commandent un câble de chargement USB-C est de $probabilityOrderUsb%")

La probabilité que les prochains clients commandent un câble de chargement USB-C est de 0.11722237088573723%


usbOrders: Long = 21903L
totalOrders: Long = 186850L
probabilityOrderUsb: Double = 0.11722237088573723

8. Quelle est la probabilité que les prochains clients commandent un iPhone ?

In [19]:
// Filtrer les commandes d'Iphone
val iphoneOrders = newdf1.filter(newdf1("Product") === "iphone").count

// Calculer le nombre total de commandes
val totalIphone = newdf1.count()

// Calculer la probabilité
val probabilityOrderIphone = iphoneOrders.toDouble / totalIphone.toDouble

// Afficher la probabilité
println(s"La probabilité que les prochains clients commandent un Iphone est de $probabilityOrderIphone%")

La probabilité que les prochains clients commandent un Iphone est de 0.0%


iphoneOrders: Long = 0L
totalIphone: Long = 186850L
probabilityOrderIphone: Double = 0.0

9. Quelle est la probabilité que les prochains clients commandent un téléphone Google ?

In [20]:
// Filtrer les commandes de Téléphone Google
val googlePhoneOrders = newdf1.filter(newdf1("Product") === "Google Phone").count

// Calculer le nombre total de commandes
val totalGooglePhone = newdf1.count()

// Calculer la probabilité
val probabilityOrderGooglePhone = googlePhoneOrders.toDouble / totalGooglePhone.toDouble

// Afficher la probabilité
println(s"La probabilité que les prochains clients commandent un téléphone Google est de $probabilityOrderGooglePhone%")

La probabilité que les prochains clients commandent un Iphone est de 0.02956917313352957%


googlePhoneOrders: Long = 5525L
totalGooglePhone: Long = 186850L
probabilityOrderGooglePhone: Double = 0.02956917313352957

10. Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?

In [22]:
// Filtrer les commandes des écouteurs filaires 
val wiredHeadphonesOrders = newdf1.filter(newdf1("Product") === "Wired Headphones").count

// Calculer le nombre total de commandes
val totalWiredHeadphones = newdf1.count()

// Calculer la probabilité
val probabilityWiredHeadphones = wiredHeadphonesOrders.toDouble / totalGooglePhone.toDouble

// Afficher la probabilité
println(s"La probabilité que les prochains clients commandent un écouteur filaire est de $probabilityWiredHeadphones%")

La probabilité que les prochains clients commandent un écouteur filaire est de 0.10105432164838106%


wiredHeadphonesOrders: Long = 18882L
totalWiredHeadphones: Long = 186850L
probabilityWiredHeadphones: Double = 0.10105432164838106

## Pratique

Proposer un cas d'utilisation concret de Machine Learning en utilisant cet ensemble de
données de ventes. Vous devrez identifier un problème spécifique et proposer un modèle
prédictif ou analytique qui peut être utilisé pour résoudre le problème.

### Etape a suivre

- Préparation des données : pretaitement des valeurs manquantes, en éliminant les valeurs aberrantes si nécessaire, et en transformant les variables catégorielles en variables numériques si possible.

- Séparation des données : Divisez l'ensemble de données en un ensemble d'entraînement et un ensemble de test. L'ensemble d'entraînement sera utilisé pour ajuster le modèle, tandis que l'ensemble de test sera utilisé pour évaluer les performances du modèle sur des données non vues.

- Sélection des variables : Identifier les variables pertinentes qui peuvent influencer la demande des ventes. Cela peut inclure des variables telles que la quantité commandée, le prix unitaire, les informations temporelles (mois, jour de la semaine, etc.), etc.

- Entraînement du modèle : Appliquer la régression linéaire en ajustant un modèle linéaire aux données d'entraînement. 

- Évaluation du modèle : Évaluer les performances du modèle en utilisant l'ensemble de test. Nous pouvons utiliser des métriques telles que l'erreur quadratique moyenne (RMSE), l'erreur absolue moyenne (MAE), le coefficient de détermination (R²) pour mesurer la précision et l'adéquation du modèle.

- Prédiction de la demande future : Une fois que nous avons évalué et validé notre modèle, vous pouvons l'utiliser pour prédire la demande future en fournissant les valeurs des variables indépendantes.

### Business Problem

Notre objectif est de construire un modèle permettant de prédire les performances de ventes future

### Machine learning problem

Il s'agit d'un problème de régression, car le prix des ventes est une variable continue. 

### Source de donnees

Nous avons juger pertinent d'utiliser l'ensemble des données sur les ventes provenant d'Amazon. Il contient des informations sur les ventes réalisé par Amazon, telles que Le type de produit qui a été vendu, La quantité commandée est la quantité totale d'articles commandés, Le prix de chaque produit, la date à laquelle le client demande que la commande soit expédiée et l'adresse de livraison. Ces donnees sont telechargeable sur Google Classroom. Cependant nous les avons deja telecharge et placer dans le repertoire data du cours.

### Selection empirique de features

Nous allons conserver que les colonnes qui nous semblent pertinentes pour notre analyse

In [149]:
newdf1.select(
    "Order ID",
    "Quantity Ordered",
    "Price Each",
    "Total Sales",
).show

+--------+----------------+----------+-----------+
|Order ID|Quantity Ordered|Price Each|Total Sales|
+--------+----------------+----------+-----------+
|141234.0|             1.0|     700.0|      700.0|
|141235.0|             1.0|     14.95|      14.95|
|141236.0|             2.0|     11.99|      23.98|
|141237.0|             1.0|    149.99|     149.99|
|141238.0|             1.0|     11.99|      11.99|
|141239.0|             1.0|      2.99|       2.99|
|141240.0|             1.0|    389.99|     389.99|
|141241.0|             1.0|     11.95|      11.95|
|141242.0|             1.0|     99.99|      99.99|
|141243.0|             1.0|     150.0|      150.0|
|141244.0|             1.0|     150.0|      150.0|
|141245.0|             1.0|    1700.0|     1700.0|
|141246.0|             3.0|      2.99|       8.97|
|141247.0|             1.0|    149.99|     149.99|
|141248.0|             1.0|     300.0|      300.0|
|141249.0|             1.0|    149.99|     149.99|
|141250.0|             1.0|    

### Preprocessing des donnees

#### Statistiques sommaires

In [105]:
newdf1.select(
    "Order ID",
    "Product",
    "Quantity Ordered",
    "Price Each",
    "Order Date",
    "Total Sales",
  )
  .summary()
  .show

+-------+-----------------+------------+-------------------+------------------+--------------+-----------------+
|summary|         Order ID|     Product|   Quantity Ordered|        Price Each|    Order Date|      Total Sales|
+-------+-----------------+------------+-------------------+------------------+--------------+-----------------+
|  count|           185950|      186305|             185950|            185950|        186305|           185950|
|   mean|230417.5693788653|        null| 1.1243828986286637|184.39973476743927|          null|185.4909167518412|
| stddev|51512.73710999596|        null|0.44279262402866965| 332.7313298843437|          null|332.9197713864793|
|    min|         141234.0|20in Monitor|                1.0|              2.99|01/01/19 03:07|             2.99|
|    25%|         185815.0|        null|                1.0|             11.95|          null|            11.95|
|    50%|         230355.0|        null|                1.0|             14.95|          null|  

D'après le tableau fourni, il semble y avoir des données manquantes dans la colonne "Product" et dans la colonne "Order Date". Cela est indiqué par la présence de la valeur "null" dans les statistiques sommaires pour ces deux variables.

En ce qui concerne les autres variables, telles que l'identifiant de commande ("Order ID"), la quantité commandée ("Quantity Ordered"), le prix unitaire ("Price Each") et le chiffre d'affaires total ("Total Sales"), toutes les valeurs semblent être disponibles et complètes, car il n'y a pas de mention de "null" dans les statistiques sommaires pour ces variables.

Cependant, il convient de noter qu'il y a une différence de comptage entre le nombre total d'observations pour "Order ID" (186 305) et le nombre d'observations pour "Total Sales" (185 950). Cela suggère qu'il pourrait y avoir des valeurs manquantes ou des données non disponibles pour certaines commandes dans la colonne "Total Sales".

Pour une évaluation plus précise de la présence de données manquantes, il serait recommandé d'examiner de plus près les données brutes pour identifier les valeurs manquantes ou les motifs éventuels de données manquantes.

#### Gestion des valeurs manquantes

In [144]:
//Verification du type des variables
newdf1.schema

res143: types.StructType = StructType(
  StructField("Order ID", DoubleType, true, {}),
  StructField("Product", StringType, true, {}),
  StructField("Quantity Ordered", DoubleType, true, {}),
  StructField("Price Each", DoubleType, true, {}),
  StructField("Order Date", StringType, true, {}),
  StructField("Purchase Address", StringType, true, {}),
  StructField("Total Sales", DoubleType, true, {}),
  StructField("Address", ArrayType(StringType, true), true, {}),
  StructField("Ville", StringType, true, {})
)

#### Identification des colonnes manquantes

In [107]:
val rowCount = newdf1.count()
println(s"Le nombre total de lignes dans le DataFrame est : $rowCount")

Le nombre total de lignes dans le DataFrame est : 186850


rowCount: Long = 186850L

In [120]:
newdf.columns.map(c => (c, newdf.filter(col(c).isNull || col(c).isNaN).count()))
    .filter(_._2 > 0)

res119: Array[(String, Long)] = Array(
  ("Order ID", 900L),
  ("Product", 545L),
  ("Quantity Ordered", 900L),
  ("Price Each", 900L),
  ("Order Date", 545L),
  ("Purchase Address", 545L),
  ("Total Sales", 900L)
)

Les colonnes: Product,Order Date,Purchase Address et Total Sales ont des valeurs manquantes

##### Imputation

In [152]:
//Ajoutez une variable fictive si nous voulons imputer une valeur.
val missingCols = newdf.columns.map(c => (c, newdf.filter(col(c).isNull || col(c).isNaN).count()))
    .filter(_._2 > 0)
    .map(_._1)

import org.apache.spark.sql.functions.when

for (c <- missingCols)
  doublesDF = newdf.withColumn(c + "_na", when(col(c).isNull, 1.0).otherwise(0.0))

missingCols: Array[String] = Array(
  "Order ID",
  "Product",
  "Quantity Ordered",
  "Price Each",
  "Order Date",
  "Purchase Address",
  "Total Sales"
)
import org.apache.spark.sql.functions.when



Ajoutons de nouvelles colonnes au DataFrame doublesDF pour chaque colonne dans missingCols afin de marquer les valeurs nulles (null) avec 1.0 et les valeurs non nulles avec 0.0.

In [153]:
import org.apache.spark.sql.functions.when

for (c <- missingCols)
  doublesDF = newdf.withColumn(c + "_na", when(col(c).isNull, 1.0).otherwise(0.0))
doublesDF.printSchema

root
 |-- Order ID: double (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: double (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Total Sales: double (nullable = true)
 |-- Total Sales_na: double (nullable = false)



import org.apache.spark.sql.functions.when



Imputons par la medianne:

In [225]:
import org.apache.spark.ml.feature.Imputer

val imputeCols = Array("Order ID","Quantity Ordered","Price Each", "Total Sales")

val imputer = new Imputer()
  .setStrategy("median")
  .setInputCols(imputeCols)
  .setOutputCols(imputeCols)

val imputedDF = imputer.fit(newdf).transform(newdf)

import org.apache.spark.ml.feature.Imputer


imputeCols: Array[String] = Array(
  "Order ID",
  "Quantity Ordered",
  "Price Each",
  "Total Sales"
)
imputer: Imputer = imputer_1d93e25c9d0c
imputedDF: DataFrame = [Order ID: double, Product: string ... 5 more fields]

In [ ]:
imputedDF.columns.map(c => (c, imputedDF.filter(col(c).isNull || col(c).isNaN).count()))
    .filter(_._2 > 0)

##### Separtion des données en donnée d’entraînement(training) et des donnée de test(test)

L'ensemble d'entraînement sera utilisé pour entraîner le modèle, tandis que l'ensemble de test sera utilisé pour évaluer les performances du modèle sur des données non vues. Ainsi nous allons garder:<br>
- 80% pour le training et réservons <br>
- 20% de nos données pour le test. <br><br>
Nous utiliserons la méthode `randomSplit()` pour cela:

In [297]:
val Array(trainDF, testDF) = imputedDF.randomSplit(Array(.8, .2), seed=28)
println(f"Il y'a ${trainDF.cache().count()} lignes dans le training set, et ${testDF.cache().count()} dans le test set")

23/05/26 23:03:05 WARN CacheManager: Asked to cache already cached data.
23/05/26 23:03:05 WARN CacheManager: Asked to cache already cached data.


Il y'a 149410 lignes dans le training set, et 37440 dans le test set


trainDF: Dataset[Row] = [Order ID: double, Product: string ... 5 more fields]
testDF: Dataset[Row] = [Order ID: double, Product: string ... 5 more fields]

##### Preparation des donnees

In [301]:
import org.apache.spark.ml.feature.VectorAssembler

val vecAssembler = new VectorAssembler()
  .setInputCols(Array("Quantity Ordered"))
  .setOutputCol("features")

val vecTrainDF = vecAssembler.transform(trainDF)

vecTrainDF
  .select("Quantity Ordered","features","Total Sales")
  .show(10)

+----------------+--------+-----------+
|Quantity Ordered|features|Total Sales|
+----------------+--------+-----------+
|             1.0|   [1.0]|      700.0|
|             1.0|   [1.0]|      14.95|
|             1.0|   [1.0]|      11.99|
|             1.0|   [1.0]|     389.99|
|             1.0|   [1.0]|      99.99|
|             1.0|   [1.0]|      150.0|
|             1.0|   [1.0]|      150.0|
|             1.0|   [1.0]|     1700.0|
|             1.0|   [1.0]|     149.99|
|             1.0|   [1.0]|      300.0|
+----------------+--------+-----------+
only showing top 10 rows



import org.apache.spark.ml.feature.VectorAssembler


vecAssembler: VectorAssembler = vecAssembler_eddb80c8b8a8
vecTrainDF: DataFrame = [Order ID: double, Product: string ... 6 more fields]

##### Création du modèle

Dans ce projet, nous voulons ajuster un modèle de régression linéaire pour prédire le montant des ventes en fonction du nombre de commande.

###### Estimation

In [302]:
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression()
  .setFeaturesCol("features")
  .setLabelCol("Total Sales")

val lrModel = lr.fit(vecTrainDF)

23/05/26 23:07:38 WARN Instrumentation: [49cb7b90] regParam is zero, which might cause numerical instability and overfitting.


import org.apache.spark.ml.regression.LinearRegression


lr: LinearRegression = linReg_bb59fe09d259
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_bb59fe09d259

Examinons le modele

In [303]:
val m = lrModel.coefficients(0)
val b = lrModel.intercept

println(f"La formule de la ligne de régression linéaire est le prix = $m%1.2f*Total Sales + $b%1.2f")
println("*-"*80)

La formule de la ligne de régression linéaire est le prix = -104.59*Total Sales + 302.66
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-


m: Double = -104.58567230574222
b: Double = 302.65986592143605

###### Création d'un pipeline

In [304]:
import org.apache.spark.ml.Pipeline

val pipeline = new Pipeline().setStages(Array(vecAssembler, lr))
val pipelineModel = pipeline.fit(trainDF)

23/05/26 23:07:51 WARN Instrumentation: [a6ad0632] regParam is zero, which might cause numerical instability and overfitting.


import org.apache.spark.ml.Pipeline


pipeline: Pipeline = pipeline_e0383847ed70
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_e0383847ed70

In [305]:
val predDF = pipelineModel.transform(testDF)

predDF
  .select("Quantity Ordered","features", "Total Sales", "prediction")
  .show(10)

+----------------+--------+-----------+-------------------+
|Quantity Ordered|features|Total Sales|         prediction|
+----------------+--------+-----------+-------------------+
|             2.0|   [2.0]|      23.98|  93.48852130995161|
|             1.0|   [1.0]|     149.99| 198.07419361569384|
|             1.0|   [1.0]|       2.99| 198.07419361569384|
|             1.0|   [1.0]|      11.95| 198.07419361569384|
|             3.0|   [3.0]|       8.97|-11.097150995790628|
|             1.0|   [1.0]|      11.95| 198.07419361569384|
|             1.0|   [1.0]|       3.84| 198.07419361569384|
|             1.0|   [1.0]|      11.95| 198.07419361569384|
|             1.0|   [1.0]|      99.99| 198.07419361569384|
|             1.0|   [1.0]|      150.0| 198.07419361569384|
+----------------+--------+-----------+-------------------+
only showing top 10 rows



predDF: DataFrame = [Order ID: double, Product: string ... 7 more fields]

###### Évaluation du modèle 

Nous utiliserons l'erreur quadratique moyenne ($RMSE$) et $R^2$ (prononcé "*R-scared*") pour évaluer la performance de notre modèle.<br>
$$RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}\Big({y_i -\hat{y_i}}\Big)^2}$$

In [306]:
import org.apache.spark.ml.evaluation.RegressionEvaluator 

val regressionEvaluator = new RegressionEvaluator()
  .setPredictionCol("prediction")
  .setLabelCol("Total Sales")
  .setMetricName("rmse")

val rmse = regressionEvaluator.evaluate(predDF) 

import org.apache.spark.ml.evaluation.RegressionEvaluator 


regressionEvaluator: RegressionEvaluator = regEval_440f84b5695a
rmse: Double = 325.50468389560524

###### Interpretation de la RMSE

Nous utiliserons la méthode des $R^2$ pour interpréter cette valeur

In [307]:
val r2 = regressionEvaluator
  .setMetricName("r2")
  .evaluate(predDF) 

println(s"R2 est $r2")

R2 est 0.019139918581469795


r2: Double = 0.019139918581469795

###### Modele de reference

Calculons le prix moyen sur le training set, et utilisons cela comme colonne de prédiction pour notre ensemble de données de test, puis évaluons le résultat:

In [308]:
import org.apache.spark.sql.functions.{avg, lit}

val avgPrice = trainDF
  .select(avg("Total Sales"))
  .first()
  .getDouble(0)

val predBaseLineDF = testDF.withColumn("avgPrediction", lit(avgPrice))

predBaseLineDF
  .select("Quantity Ordered", "Total Sales", "avgPrediction")
  .show(10)

+----------------+-----------+------------------+
|Quantity Ordered|Total Sales|     avgPrediction|
+----------------+-----------+------------------+
|             2.0|      23.98|185.10825821565862|
|             1.0|     149.99|185.10825821565862|
|             1.0|       2.99|185.10825821565862|
|             1.0|      11.95|185.10825821565862|
|             3.0|       8.97|185.10825821565862|
|             1.0|      11.95|185.10825821565862|
|             1.0|       3.84|185.10825821565862|
|             1.0|      11.95|185.10825821565862|
|             1.0|      99.99|185.10825821565862|
|             1.0|      150.0|185.10825821565862|
+----------------+-----------+------------------+
only showing top 10 rows



import org.apache.spark.sql.functions.{avg, lit}


avgPrice: Double = 185.10825821565862
predBaseLineDF: DataFrame = [Order ID: double, Product: string ... 6 more fields]

In [310]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val regressionMeanEvaluator = new RegressionEvaluator()
  .setPredictionCol("avgPrediction")
  .setLabelCol("Total Sales")
  .setMetricName("rmse")

val rmseBaseLine = regressionMeanEvaluator.evaluate(predBaseLineDF)
println (f"Le RMSE pour prédire le prix moyen est: $rmseBaseLine%1.2f")

Le RMSE pour prédire le prix moyen est: 328.67


import org.apache.spark.ml.evaluation.RegressionEvaluator


regressionMeanEvaluator: RegressionEvaluator = regEval_c6a62b22616a
rmseBaseLine: Double = 328.6724875297801

In [311]:
val r2BaseLine = regressionMeanEvaluator
  .setMetricName("r2")
  .evaluate(predBaseLineDF) 

println(s"R2 BaseLine est $r2BaseLine")

R2 BaseLine est -4.439271514700138E-5


r2BaseLine: Double = -4.439271514700138E-5